In [577]:
import torch
from torch import nn 
from torch import optim
from river import compose, metrics, preprocessing, stream, anomaly, linear_model, datasets, compose
from river import feature_extraction as fx
from IncrementalTorch.anomaly.anomaly import TorchAE, SklearnAnomalyDetector
from tqdm import tqdm
import river  
import torchvision
import numpy as np
import pandas as pd
from pprint import pprint

from OnlineTorch.classifier import PyTorch2RiverClassifier
from torch import nn, optim

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import SGDOneClassSVM
from sklearn.cluster import k_means
from sklearn.decomposition import IncrementalPCA

N_SAMPLES = 1_000
SEED = 42
track_name = "RBF"
#LOSS = nn.BCELoss
LOSS = nn.L1Loss
OPTIMIZER = optim.AdamW
BATCH_SIZE=10
LEARNING_RATE=1e-3
LATENT_DIM = 1
rocauc = river.metrics.ROCAUC()
rocauc=0

##max 5% with 10000 samples
anom_percentage = 50
num_samples = 1000


#F1,F2,F0,5 interessant?


In [578]:
#evaluation=pd.DataFrame(columns=['model','anom_percentage','num_samples','Loss','Optimizer','Batch_size','Learning_rate','Latent_dim','ROC','Precision_Recall_Curve','FP','TP','FN','TN'])


In [579]:
'''evaluation.append({
    'model':1,
    'anom_percentage':anom_percentage,
    'num_samples':num_samples,
    'Loss':LOSS,
    'Optimizer':OPTIMIZER,
    'Batch_size':BATCH_SIZE,
    'Learning_rate':LEARNING_RATE,
    'Latent_dim': LATENT_DIM,
    'ROC':rocauc,
    #'Precision_Recall_Curve':45,
    #'FP':5,
    #'TP': 4,
    #'FN':8,
    #'TN':10

},ignore_index=True)
'''

"evaluation.append({\n    'model':1,\n    'anom_percentage':anom_percentage,\n    'num_samples':num_samples,\n    'Loss':LOSS,\n    'Optimizer':OPTIMIZER,\n    'Batch_size':BATCH_SIZE,\n    'Learning_rate':LEARNING_RATE,\n    'Latent_dim': LATENT_DIM,\n    'ROC':rocauc,\n    #'Precision_Recall_Curve':45,\n    #'FP':5,\n    #'TP': 4,\n    #'FN':8,\n    #'TN':10\n\n},ignore_index=True)\n"

## Dataset

In [580]:
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=SEED)

In [581]:

num_anom = int(num_samples*(anom_percentage/100))
num_clean = int(num_samples-num_anom)

data_stream = stream.shuffle(river.datasets.CreditCard(), N_SAMPLES, seed=SEED)
data1 = pd.DataFrame(data=data_stream)
df_test= pd.DataFrame.from_dict(data1)

anoms=df_test[df_test[1]==1].iloc[0:num_anom]
clean = df_test[df_test[1]==0].iloc[0:num_clean]
frames = [anoms,clean]
print('Stream contains {} anomalies and {} no-anomalies'.format(anoms[0].count(),clean[0].count()))
final_set = pd.concat(frames,ignore_index=True)

data_stream1 = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)

Stream contains 492 anomalies and 500 no-anomalies


In [582]:
'''

    anom_percentage = 20
    num_samples = 2500
    
    num_anom = int(num_samples*(anom_percentage/100))
    num_clean = int(num_samples-num_anom)

    data_stream = stream.shuffle(river.datasets.CreditCard(), N_SAMPLES, seed=SEED)
    data1 = pd.DataFrame(data=data_stream)
    df_test= pd.DataFrame.from_dict(data1)

    anoms=df_test[df_test[1]==1].iloc[0:num_anom]
    clean = df_test[df_test[1]==0].iloc[0:num_clean]
    frames = [anoms,clean]
    print('Stream contains {} anomalies and {} no-anomalies'.format(anoms[0].count(),clean[0].count()))
    final_set = pd.concat(frames,ignore_index=True)

    data_stream1 = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)


    model3 = compose.Pipeline(
        preprocessing.MinMaxScaler(),
        anomaly.HalfSpaceTrees(seed=SEED)
    )
    rocauc= river.metrics.ROCAUC()
    #data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)
    data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
    for x, y in data_stream:
        model3.learn_one(x)
        y_pred= model3.score_one(x)
        #print(y_pred)
        rocauc.update(y,y_pred)
    #evaluation.append(rocauc)
    rocauc
    evaluation=evaluation.append({
        'model':'HalfSpaceTrees_min_max_scaler',
        'anom_percentage':anom_percentage,
        'num_samples':num_samples,
        'Loss':LOSS,
        'Optimizer':OPTIMIZER,
        'Batch_size':BATCH_SIZE,
        'Learning_rate':LEARNING_RATE,
        'Latent_dim': LATENT_DIM,
        'ROC':rocauc,
        #'Precision_Recall_Curve':45,
        #'FP':5,
        #'TP': 4,
        #'FN':8,
        #'TN':10

    },ignore_index=True)
    '''

"\n\n    anom_percentage = 20\n    num_samples = 2500\n    \n    num_anom = int(num_samples*(anom_percentage/100))\n    num_clean = int(num_samples-num_anom)\n\n    data_stream = stream.shuffle(river.datasets.CreditCard(), N_SAMPLES, seed=SEED)\n    data1 = pd.DataFrame(data=data_stream)\n    df_test= pd.DataFrame.from_dict(data1)\n\n    anoms=df_test[df_test[1]==1].iloc[0:num_anom]\n    clean = df_test[df_test[1]==0].iloc[0:num_clean]\n    frames = [anoms,clean]\n    print('Stream contains {} anomalies and {} no-anomalies'.format(anoms[0].count(),clean[0].count()))\n    final_set = pd.concat(frames,ignore_index=True)\n\n    data_stream1 = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)\n\n\n    model3 = compose.Pipeline(\n        preprocessing.MinMaxScaler(),\n        anomaly.HalfSpaceTrees(seed=SEED)\n    )\n    rocauc= river.metrics.ROCAUC()\n    #data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)\n    data_stream = stream

In [583]:
evaluation

,model,anom_percentage,num_samples,Loss,Optimizer,Batch_size,Learning_rate,Latent_dim,ROC,Precision_Recall_Curve,FP,TP,FN,TN
0,undercomplete_ae_model,5,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.843084,NaN,NaN,NaN,NaN,NaN
1,undercomplete_stacked_ae_model,5,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.836194,NaN,NaN,NaN,NaN,NaN
2,HalfSpaceTrees,5,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.603265,NaN,NaN,NaN,NaN,NaN
3,undercomplete_ae_model,4,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.849106,NaN,NaN,NaN,NaN,NaN
4,undercomplete_stacked_ae_model,4,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.839135,NaN,NaN,NaN,NaN,NaN
5,HalfSpaceTrees,4,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.595489,NaN,NaN,NaN,NaN,NaN
6,undercomplete_ae_model,3,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.874875,NaN,NaN,NaN,NaN,NaN
7,undercomplete_stacked_ae_model,3,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.877815,NaN,NaN,NaN,NaN,NaN
8,HalfSpaceTrees,3,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.55782,NaN,NaN,NaN,NaN,NaN
9,undercomplete_ae_model,2,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.900002,NaN,NaN,NaN,NaN,NaN


# Autoencoder
## Undercomplete Autoencoder
### Softmax

In [584]:
'''def undercomplete_ae_sm(n_features, latent_dim=1):
    net = nn.Sequential(
        nn.Dropout(),
        nn.Linear(n_features, 20), 
        nn.LeakyReLU(),
        nn.Linear(20, latent_dim),
        nn.LeakyReLU(),
        nn.Linear(latent_dim, 20),
        nn.LeakyReLU(), 
        nn.Linear(20, n_features),
        nn.Linear(n_features,1),
        nn.Softmax()
    )
    return net
    '''

'def undercomplete_ae_sm(n_features, latent_dim=1):\n    net = nn.Sequential(\n        nn.Dropout(),\n        nn.Linear(n_features, 20), \n        nn.LeakyReLU(),\n        nn.Linear(20, latent_dim),\n        nn.LeakyReLU(),\n        nn.Linear(latent_dim, 20),\n        nn.LeakyReLU(), \n        nn.Linear(20, n_features),\n        nn.Linear(n_features,1),\n        nn.Softmax()\n    )\n    return net\n    '

In [585]:
'''undercomplete_ae_sm_model = compose.Pipeline(
    preprocessing.StandardScaler(),
    PyTorch2RiverClassifier(
                build_fn = undercomplete_ae_sm,
                loss_fn = LOSS,
                optimizer_fn = OPTIMIZER,
                #batch_size=BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                seed=SEED
    )
)
'''

'undercomplete_ae_sm_model = compose.Pipeline(\n    preprocessing.StandardScaler(),\n    PyTorch2RiverClassifier(\n                build_fn = undercomplete_ae_sm,\n                loss_fn = LOSS,\n                optimizer_fn = OPTIMIZER,\n                #batch_size=BATCH_SIZE,\n                learning_rate=LEARNING_RATE,\n                seed=SEED\n    )\n)\n'

In [586]:
'''#supervised learning approach with Softmax function --> Proba gets predicted, ROC way worse
rocauc = river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    y_pred = undercomplete_ae_sm_model.predict_proba_one(x) #ruft learn_unsupervised auf, müssen wir learn_one dann überhaupt auch aufrufen?
    undercomplete_ae_sm_model.learn_one(x, y) #undercomplete_ae_sm_model.learn_one ist supervised Ansatz, wir wollen Unsupervised
    rocauc.update(y, y_pred)
rocauc
'''

'#supervised learning approach with Softmax function --> Proba gets predicted, ROC way worse\nrocauc = river.metrics.ROCAUC()\n#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)\ndata_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)\nfor x, y in data_stream:\n    y_pred = undercomplete_ae_sm_model.predict_proba_one(x) #ruft learn_unsupervised auf, müssen wir learn_one dann überhaupt auch aufrufen?\n    undercomplete_ae_sm_model.learn_one(x, y) #undercomplete_ae_sm_model.learn_one ist supervised Ansatz, wir wollen Unsupervised\n    rocauc.update(y, y_pred)\nrocauc\n'

## Undercomplete Autoencoder standard

In [587]:
def undercomplete_ae(n_features, latent_dim=1):
    net = nn.Sequential(
        nn.Dropout(),
        nn.Linear(n_features, 20),
        nn.LeakyReLU(),
        nn.Linear(20, latent_dim),
        nn.LeakyReLU(),
        nn.Linear(latent_dim, 20),
        nn.LeakyReLU(), 
        nn.Linear(20, n_features),
    )
    return net

In [588]:
undercomplete_ae_model = compose.Pipeline(
    preprocessing.MinMaxScaler(),
    TorchAE(
                build_fn = undercomplete_ae,
                loss_fn = LOSS,
                optimizer_fn = OPTIMIZER,
                #batch_size=BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                seed=SEED
    )
)

In [589]:
rocauc= river.metrics.ROCAUC()
#f1 = river.metrics.F1()
#cm=river.metrics.ConfusionMatrix()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    undercomplete_ae_model.learn_one(x)
    y_pred= undercomplete_ae_model.score_one(x)
    #print(y_pred)
    rocauc.update(y,y_pred)
    #f1.update(y,y_pred)
    #cm.update(y,y_pred)
    #undercomplete_ae_model.learn_one(x,y_pred)
    #undercomplete_ae_model.learn_one(x,learn_unsupervised=True)
    #y_pred = undercomplete_ae_sm_model.predict_proba_one(x) #ruft learn_unsupervised auf, müssen wir learn_one dann überhaupt auch aufrufen?
    #undercomplete_ae_model.learn_one(x, y)
    #rocauc.update(y, y_pred)
rocauc
evaluation=evaluation.append({
    'model':'undercomplete_ae_model',
    'anom_percentage':anom_percentage,
    'num_samples':num_samples,
    'Loss':LOSS,
    'Optimizer':OPTIMIZER,
    'Batch_size':BATCH_SIZE,
    'Learning_rate':LEARNING_RATE,
    'Latent_dim': LATENT_DIM,
    'ROC':rocauc,
    #'Precision_Recall_Curve':45,
    #'FP':5,
    #'TP': 4,
    #'FN':8,
    #'TN':10

},ignore_index=True)
#f1

In [590]:
#f1

In [591]:
#cm

In [592]:
evaluation

,model,anom_percentage,num_samples,Loss,Optimizer,Batch_size,Learning_rate,Latent_dim,ROC,Precision_Recall_Curve,FP,TP,FN,TN
0,undercomplete_ae_model,5,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.843084,NaN,NaN,NaN,NaN,NaN
1,undercomplete_stacked_ae_model,5,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.836194,NaN,NaN,NaN,NaN,NaN
2,HalfSpaceTrees,5,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.603265,NaN,NaN,NaN,NaN,NaN
3,undercomplete_ae_model,4,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.849106,NaN,NaN,NaN,NaN,NaN
4,undercomplete_stacked_ae_model,4,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.839135,NaN,NaN,NaN,NaN,NaN
5,HalfSpaceTrees,4,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.595489,NaN,NaN,NaN,NaN,NaN
6,undercomplete_ae_model,3,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.874875,NaN,NaN,NaN,NaN,NaN
7,undercomplete_stacked_ae_model,3,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.877815,NaN,NaN,NaN,NaN,NaN
8,HalfSpaceTrees,3,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.55782,NaN,NaN,NaN,NaN,NaN
9,undercomplete_ae_model,2,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.900002,NaN,NaN,NaN,NaN,NaN


## Stacked Autoencoder

In [593]:
def stacked_ae(n_features, latent_dim=1):
    net = nn.Sequential(
        nn.Dropout(),
        nn.Linear(n_features, 20),
        nn.LeakyReLU(),
        nn.Linear(20, 15),
        nn.LeakyReLU(),        
        nn.Linear(15, 10),
        nn.LeakyReLU(),        
        nn.Linear(10, latent_dim),
        nn.LeakyReLU(),
        nn.Linear(latent_dim, 10),
        nn.LeakyReLU(),
        nn.Linear(10, 15),
        nn.LeakyReLU(),
        nn.Linear(15, 20),
        nn.LeakyReLU(),               
        nn.Linear(20, n_features),
    )
    return net

In [594]:
stacked_ae_model = compose.Pipeline(
    preprocessing.MinMaxScaler(),
    TorchAE(
                build_fn = stacked_ae,
                loss_fn = LOSS,
                optimizer_fn = OPTIMIZER,
                #batch_size=BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                seed=SEED
    )
)

In [595]:
rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    stacked_ae_model.learn_one(x)
    y_pred= stacked_ae_model.score_one(x)
    rocauc.update(y,y_pred)
rocauc
evaluation=evaluation.append({
    'model':'undercomplete_stacked_ae_model',
    'anom_percentage':anom_percentage,
    'num_samples':num_samples,
    'Loss':LOSS,
    'Optimizer':OPTIMIZER,
    'Batch_size':BATCH_SIZE,
    'Learning_rate':LEARNING_RATE,
    'Latent_dim': LATENT_DIM,
    'ROC':rocauc,
    #'Precision_Recall_Curve':45,
    #'FP':5,
    #'TP': 4,
    #'FN':8,
    #'TN':10

},ignore_index=True)

In [596]:
#evaluation

## Sparse Autoencoder

# Baselines
## PCA (Incremental)

In [597]:
ipca = IncrementalPCA(n_components=LATENT_DIM, batch_size=1)

In [598]:
model6=compose.Pipeline(
    preprocessing.MinMaxScaler(),
    ipca #IncrementalPCA(n_components=LATENT_DIM, batch_size=1)
)

In [599]:
model7=compose.Pipeline(
    preprocessing.MinMaxScaler(),
    SklearnAnomalyDetector(IncrementalPCA(n_components=10, batch_size=1))
)

In [600]:
#rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=SEED)
#for x, y in data_stream:
#    print(x.values())

In [601]:
#data_stream = stream.shuffle(river.datasets.CreditCard().take(20000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
#data_stream = list(data_stream)

data1 = pd.DataFrame(data=data_stream)
#data2 = pd.DataFrame.from_dict(data=data1,orient='index')

In [602]:
#test=data1[0].to_dict()
#df_test=pd.DataFrame.from_dict(test, orient='index')

In [603]:
#df_test

In [604]:
#pd.DataFrame.from_dict({(i,j): data1[i][j] 
#                           for i in data1.keys() 
#                           for j in data1[i].keys()},
#                       orient='index')

In [605]:
#data_stream = stream.shuffle(river.datasets.CreditCard().take(20000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
data1 = pd.DataFrame(data=data_stream)
df_test= pd.DataFrame.from_dict(data1)


data_stream1 = stream.shuffle(df_test.itertuples(index=False),N_SAMPLES, seed=SEED)

In [606]:
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
#df=pd.read_csv(r'C:\Users\Manuel\river_data\CreditCard\creditcard.csv')

In [607]:
rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(5000), N_SAMPLES, seed=40)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
MSE_score_arr = []
values = []
for x, y in data_stream:
    #print(pd.DataFrame.from_dict(x))
    ipca = ipca.partial_fit(pd.DataFrame.from_dict([x]))
    x_trans = ipca.transform(pd.DataFrame.from_dict([x]))
    inverse_trans= ipca.inverse_transform(x_trans)
    MSE_score = ((np.array(pd.DataFrame.from_dict([x]))-np.array([inverse_trans]))**2).sum()
    MSE_score_arr.append(MSE_score)
    values.append(y)
    #model6.learn_one(x)
    #y_pred= model6.score_one(x)
    #rocauc.update(y,y_pred)
#rocauc

C:\Users\Manuel\Anaconda3\envs\rwch\lib\site-packages\sklearn\decomposition\_incremental_pca.py:337: RuntimeWarning: invalid value encountered in true_divide
  explained_variance = S ** 2 / (n_total_samples - 1)
C:\Users\Manuel\Anaconda3\envs\rwch\lib\site-packages\sklearn\decomposition\_incremental_pca.py:338: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
C:\Users\Manuel\Anaconda3\envs\rwch\lib\site-packages\sklearn\decomposition\_incremental_pca.py:348: RuntimeWarning: Mean of empty slice.
  self.noise_variance_ = explained_variance[self.n_components_ :].mean()
C:\Users\Manuel\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [608]:
values_df = pd.Series(data=values)

In [609]:
loss = pd.Series(data=MSE_score_arr)
loss = (loss-np.min(loss))/(np.max(loss)-np.min(loss))

In [610]:
loss.sum()

12.014285618945905

In [611]:
rocauc = metrics.ROCAUC()
for yt, yp in zip(values, loss):
    #print(yt, yp)
    rocauc = rocauc.update(yt, yp)
rocauc
#evaluation.append(rocauc)

ROCAUC: 0.508268

## OneClassSVM

In [612]:
river.__version__  #0.9.0 required for OneClassSVM

'0.8.0'

In [613]:
'''#funktioniert nur mit river=0.9, allerdings muss dann git repo geupdatet werden, da anomalie.anomaliedetector klasse nicht mehr in base sondern in anomly ist --> Wheels können nicht mehr so gebaut werden wie bisher
model4 = compose.Pipeline(
    preprocessing.StandardScaler(),
    #fx.RBFSampler(),    
    anomaly.QuantileThresholder(
        anomaly.OneClassSVM(),
        q=0.97 #q Anpassung viele Auswirkungen
    )
)
'''

'#funktioniert nur mit river=0.9, allerdings muss dann git repo geupdatet werden, da anomalie.anomaliedetector klasse nicht mehr in base sondern in anomly ist --> Wheels können nicht mehr so gebaut werden wie bisher\nmodel4 = compose.Pipeline(\n    preprocessing.StandardScaler(),\n    #fx.RBFSampler(),    \n    anomaly.QuantileThresholder(\n        anomaly.OneClassSVM(),\n        q=0.97 #q Anpassung viele Auswirkungen\n    )\n)\n'

In [614]:
'''rocauc= river.metrics.ROCAUC()
data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)
for x, y in data_stream:
    model4.learn_one(x)
    y_pred= model4.score_one(x)
    rocauc.update(y,y_pred)
rocauc
'''

'rocauc= river.metrics.ROCAUC()\ndata_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)\nfor x, y in data_stream:\n    model4.learn_one(x)\n    y_pred= model4.score_one(x)\n    rocauc.update(y,y_pred)\nrocauc\n'

## HalfSpaceTrees

In [615]:
model3 = compose.Pipeline(
    preprocessing.MinMaxScaler(),
    anomaly.HalfSpaceTrees(seed=SEED)
)

In [616]:
rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    model3.learn_one(x)
    y_pred= model3.score_one(x)
    #print(y_pred)
    rocauc.update(y,y_pred)
#evaluation.append(rocauc)
rocauc
evaluation=evaluation.append({
    'model':'HalfSpaceTrees_min_max_scaler',
    'anom_percentage':anom_percentage,
    'num_samples':num_samples,
    'Loss':LOSS,
    'Optimizer':OPTIMIZER,
    'Batch_size':BATCH_SIZE,
    'Learning_rate':LEARNING_RATE,
    'Latent_dim': LATENT_DIM,
    'ROC':rocauc,
    #'Precision_Recall_Curve':45,
    #'FP':5,
    #'TP': 4,
    #'FN':8,
    #'TN':10

},ignore_index=True)

In [617]:
evaluation

,model,anom_percentage,num_samples,Loss,Optimizer,Batch_size,Learning_rate,Latent_dim,ROC,Precision_Recall_Curve,FP,TP,FN,TN
0,undercomplete_ae_model,5,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.843084,NaN,NaN,NaN,NaN,NaN
1,undercomplete_stacked_ae_model,5,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.836194,NaN,NaN,NaN,NaN,NaN
2,HalfSpaceTrees,5,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.603265,NaN,NaN,NaN,NaN,NaN
3,undercomplete_ae_model,4,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.849106,NaN,NaN,NaN,NaN,NaN
4,undercomplete_stacked_ae_model,4,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.839135,NaN,NaN,NaN,NaN,NaN
5,HalfSpaceTrees,4,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.595489,NaN,NaN,NaN,NaN,NaN
6,undercomplete_ae_model,3,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.874875,NaN,NaN,NaN,NaN,NaN
7,undercomplete_stacked_ae_model,3,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.877815,NaN,NaN,NaN,NaN,NaN
8,HalfSpaceTrees,3,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.55782,NaN,NaN,NaN,NaN,NaN
9,undercomplete_ae_model,2,10000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,1,0.001,1,ROCAUC: 0.900002,NaN,NaN,NaN,NaN,NaN


In [618]:
evaluation.to_csv('evaluation_1.vers')

# Random

In [619]:
dataset = datasets.synth.RandomRBF(seed_model=7, seed_sample=SEED,n_classes=10,n_features=200).take(N_SAMPLES)

def build_fn(n_features):
    net = nn.Sequential(
        nn.Linear(n_features, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 1),
        nn.Softmax()
    )
    return net

model = compose.Pipeline(
    preprocessing.StandardScaler(),
    PyTorch2RiverClassifier(
                build_fn=build_fn,
                loss_fn=nn.BCELoss,
                optimizer_fn=optim.Adam,
                batch_size=1,
                learning_rate=1e-3,
    )
)

for x, y in data_stream:
    y_pred = model.predict_proba_one(x)
    model.learn_one(x, y)
    rocauc.update(y, y_pred)
rocauc

ROCAUC: 0.688553